In [1]:
!pip install lckr_jupyterlab_variableinspector

In [2]:
from datetime import timedelta
import random
from scipy.stats import pareto
from scipy.stats import expon
from scipy.stats import lognorm
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import os

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [4]:
TPLL = None
TPS_GET = None
TPS_POST = None
NT_GET = None
NT_POST = None
STLL_GET = None
STLL_POST = None
STS_GET = None
STS_POST = None
STO_GET = None
STO_POST = None
STA_GET = None
STA_POST = None
CS = None
CC = None
T = None
TF = None
ITO_GET = None
ITO_POST = None
N = None
M = None
HV = None
resultados = []

In [5]:
def CI():
    global TPLL, TPS_GET, TPS_POST
    global NT_GET, NT_POST
    global STLL_GET, STLL_POST
    global STS_GET, STS_POST
    global STO_GET, STO_POST
    global CS, CC
    global T, TF
    global ITO_GET, ITO_POST
    global STA_GET, STA_POST
    global HV
    global Ti

    Ti = timedelta(hours=6)

    HV = timedelta.max

    TPLL = timedelta(hours=6)
    TPLL_POST = timedelta(hours=6)
    TPLL_GET = timedelta(hours=6)
    TPS_GET = [HV for _ in range(N)]
    TPS_POST = [HV for _ in range(M)]

    NT_GET = 0
    NT_POST = 0

    STLL_GET = timedelta(0)
    STLL_POST = timedelta(0)
    STS_GET = timedelta(0)
    STS_POST = timedelta(0)
    STA_GET = [timedelta(0) for _ in range(N)]
    STA_POST = [timedelta(0) for _ in range(M)]
    
    STO_GET = [timedelta(0) for _ in range(N)]
    STO_POST = [timedelta(0) for _ in range(M)]

    ITO_GET = [timedelta(hours=6) for _ in range(N)]
    ITO_POST = [timedelta(hours=6) for _ in range(M)]

    CS = 0
    CC = 0
    
    T = timedelta(0)
    TF = timedelta(hours=23)

In [6]:
def loggear(texto):
    with open('log.txt', 'a') as f:
        f.write(texto + '\n')

In [7]:
def MENOR_TPS(TPS):
    if not TPS:
        raise ValueError("El vector TPS está vacío.")

    indice_min = 0
    valor_min = TPS[0]

    for i in range(1, len(TPS)):
        if TPS[i] < valor_min:
            valor_min = TPS[i]
            indice_min = i

    return indice_min

In [8]:
def IA_MAÑANA():
    b = 112.66
    loc = -7966
    scale = 7967

    valor_milisegundos = pareto.rvs(b, loc=loc, scale=scale)

    # print(f"IA_MAÑANA retorno: {valor_milisegundos}")

    return timedelta(milliseconds=valor_milisegundos)

In [9]:
def IA_TARDE():
    b = 58.7
    loc = -2467.2
    scale = 2468.2

    valor_milisegundos = pareto.rvs(b, loc=loc, scale=scale)

    # print(f"IA_TARDE retorno: {valor_milisegundos}")

    return timedelta(milliseconds=valor_milisegundos)

In [10]:
def IA_NOCHE():
    loc = 1
    scale = 107.37

    valor_milisegundos = expon.rvs(loc=loc, scale=scale)

    # print(f"IA_NOCHE retorno: {valor_milisegundos}")

    return timedelta(milliseconds=valor_milisegundos)

In [11]:
def TA_SIMPLES():
    global TA_Simples

    s = 0.1
    loc = -229.95
    scale = 512.89

    valor_milisegundos = lognorm.rvs(s, loc=loc, scale=scale)

    TA_Simples = timedelta(milliseconds=valor_milisegundos)

In [12]:
def TA_COMPLEJAS():
    global TA_Complejas

    s = 0.1
    loc = -398.85
    scale = 1544.11

    valor_milisegundos = lognorm.rvs(s, loc=loc, scale=scale)

    TA_Complejas = timedelta(milliseconds=valor_milisegundos)

In [13]:
def SALIDA_GET(I):
    global T, TPS_GET, CS, ITO_GET, TA_Simples, STS_GET, N, STA_GET

    STS_GET = STS_GET + T
    
    T = TPS_GET[I]

    #loggear(f"Salida GET en el servidor {I} en instante: {T}")
    CS -= 1

    # print(f"CS luego de la salida: {CS}")

    if CS >= N:
        TA_SIMPLES()
        STA_GET[I] = STA_GET[I] + TA_Simples
        TPS_GET[I] = T + TA_Simples
    else:
        ITO_GET[I] = T

        #loggear(f"INICIO TIEMPO OCIOSO DE PUESTO GET {I}: {ITO_GET[I]}")
        TPS_GET[I] = timedelta.max

    # print(f"TPS_GET del puesto {I}: {TPS_GET[I]}") 

In [14]:
def SALIDA_POST(J):
    global T, TPS_POST, CC, ITO_POST, TA_Complejas, STS_POST, M, STA_POST

    STS_POST = STS_POST + T

    T = TPS_POST[J]

    #loggear(f"Salida POST en el servidor {J} en instante: {T}")
    
    CC -= 1

    # print(f"CC luego de la salida: {CC}")

    if CC >= M:
        TA_COMPLEJAS()
        STA_POST[J] = STA_POST[J] + TA_Complejas
        TPS_POST[J] = T + TA_Complejas
    else:
        ITO_POST[J] = T

        #loggear(f"INICIO TIEMPO OCIOSO DE PUESTO POST {J}: {ITO_POST[J]}")
        TPS_POST[J] = timedelta.max
        
    # print(f"TPS_POST del puesto {J}: {TPS_POST[J]}") 

In [15]:
def BUSCAR_PUESTO_GET():
    global TPS_GET, STO_GET

    mejor_indice = -1

    for i in range(len(TPS_GET)):
        if TPS_GET[i] == HV:
            mejor_indice = i
            return mejor_indice
            
    return mejor_indice

In [16]:
def ATENCION_GET():
    global CS, STO_GET, T, ITO_GET, TPS_GET, TA_Simples, N, STA_GET

    if CS <= N:
        X = BUSCAR_PUESTO_GET()

        #loggear(f"ITO_GET DE PUESTO {X}: {ITO_GET[X]}")

        #loggear(f"INTERVALO DE TIEMPO OCIOSO DE PUESTO {X}: {T - ITO_GET[X]}")

        STO_GET[X] = STO_GET[X] + (T - ITO_GET[X])

        #loggear(f"SUMATORIA TIEMPO OCIOSO DE PUESTO GET {X}: {STO_GET[X]}")

        TA_SIMPLES()

        STA_GET[X] = STA_GET[X] + TA_Simples

        TPS_GET[X] = T + TA_Simples

In [17]:
def BUSCAR_PUESTO_POST():
    global TPS_POST, STO_POST
    
    mejor_indice = -1

    for i in range(len(TPS_POST)):
        if TPS_POST[i] == HV:
            mejor_indice = i
            return mejor_indice

    return mejor_indice

In [18]:
def ATENCION_POST():
    global CC, STO_POST, T, ITO_POST, TPS_POST, TA_Complejas, M, STA_POST

    if CC <= M:
        
        X = BUSCAR_PUESTO_POST()

        #loggear(f"ITO_POST DE PUESTO {X}: {ITO_POST[X]}")

        #loggear(f"INTERVALO DE TIEMPO OCIOSO DE PUESTO {X}: {T - ITO_POST[X]}")

        STO_POST[X] = STO_POST[X] + (T - ITO_POST[X])

        #loggear(f"SUMATORIA TIEMPO OCIOSO DE PUESTO POST {X}: {STO_POST[X]}")

        TA_COMPLEJAS()

        STA_POST[X] = STA_POST[X] + TA_Complejas

        TPS_POST[X] = T + TA_Complejas

In [19]:
def LLEGADA():
    global T, TPLL, NT_POST, NT_GET, CC, CS, STLL_POST, STLL_GET, TPLL_POST, TPLL_GET

    R = random.random()

    T = TPLL

    #loggear(f"Instante: {T}")

    hora_actual = T.seconds // 3600

    if 6 <= hora_actual <= 10:
        IA = IA_MAÑANA()
    elif 11 <= hora_actual <= 17:
        IA = IA_TARDE()
    elif 18 <= hora_actual <= 23:
        IA = IA_NOCHE()
    else:
        IA = timedelta.max

    
    TPLL = T + IA

    #loggear(f"TPLL: {TPLL}")

    if R <= 0.7:
        NT_POST+=1
        CC += 1
        STLL_POST = STLL_POST + T

        #loggear(f"CC luego de la llegada POST: {CC}")
        ATENCION_POST()
    else:
        NT_GET+=1
        CS += 1
        STLL_GET = STLL_GET + T

        #loggear(f"CS luego de la llegada GET: {CS}")
        ATENCION_GET()

In [20]:
def RESULTADOS():
    global TRP_Complejas, TRP_Simples, PTO_GET, PTO_POST
    global STLL_POST, STLL_GET, STS_POST, STS_GET, NT_POST, NT_GET
    global STO_GET, STO_POST, STA_GET, STA_POST, N, M, Ti

    if NT_POST > 0:
        TRP_Complejas = int(((STS_POST - STLL_POST).total_seconds() * 1000)/NT_POST)
    else:
        TRP_Complejas = 0

    if NT_GET > 0:
         TRP_Simples = int(((STS_GET - STLL_GET).total_seconds() * 1000)/NT_GET)
    else:
         TRP_Simples = 0

    PTO_GET = [0 for _ in range(N)]
    PTO_POST = [0 for _ in range(M)]

    for i in range(N):
        total_sec = (T - Ti).total_seconds()
        if total_sec != 0:
            PTO_GET[i] = round((STO_GET[i].total_seconds() / total_sec)*100, 2)
        else:
            print(f"ALGO RARO PASO CON {total_sec}")
            PTO_GET[i] = 0

    for j in range(M):
        total_sec = (T - Ti).total_seconds()
        if total_sec != 0:
            PTO_POST[j] = round((STO_POST[j].total_seconds() * 100) / total_sec, 2)
        else:
            print(f"ALGO RARO PASO CON {total_sec}")
            PTO_POST[j] = 0

In [ ]:
def clasificar_tipo(row):
    if row['N'] == 3 and row['M'] == 20:
        return 'OPTIMO'
    elif row['N'] == 2 and row['M'] == 18:
        return 'ACTUAL'
    elif row['N'] == 5 and row['M'] == 19:
        return 'INTERMEDIO'
    else:
        return 'OTRO'

In [ ]:
def GENERAR_DATAFRAME():
    global N, M, TRP_Simples, TRP_Complejas, PTO_GET, PTO_POST

    datos = {
        'N': [N],
        'M': [M],
        'TRP_Simples': [TRP_Simples],
        'TRP_Complejas': [TRP_Complejas],
    }

    for i in range(5):
        datos[f'PTO_GET_{i+1}'] = [PTO_GET[i] if i < len(PTO_GET) else None]

    for i in range(20):
        datos[f'PTO_POST_{i+1}'] = [PTO_POST[i] if i < len(PTO_POST) else None]

    nuevo_df = pd.DataFrame(datos)

    nuevo_df['tipo'] = nuevo_df.apply(clasificar_tipo, axis=1)

    try:
        df_existente = pd.read_excel("resultados.xlsx")
        df_final = pd.concat([df_existente, nuevo_df], ignore_index=True)
    except FileNotFoundError:
        df_final = nuevo_df

    df_final.to_excel("resultados.xlsx", index=False)

    return nuevo_df


In [24]:
def EJECUCION():
    global T, TPLL, TPS_GET, TPS_POST, HV, CS, CC
    
    while True:
        I = MENOR_TPS(TPS_GET)
        J = MENOR_TPS(TPS_POST)

        if TPS_GET[I] <= TPS_POST[J]:
            if TPS_GET[I] <= TPLL:
                SALIDA_GET(I)
            else:
                LLEGADA()
        else:
            if TPS_POST[J] <= TPLL:
                SALIDA_POST(J)
            else:
                LLEGADA()

        if not (T <= TF):
            break

    # print(f'CS: {CS}')
    # print(f'CC: {CC}')
    
    if not (CS == 0 and CC == 0):
         TPLL = HV
         EJECUCION()

In [25]:
def SIMULACION():
    CI()

    EJECUCION()
        
    RESULTADOS()    

In [26]:
global N, M

In [27]:
def RUN(n,m):
        global N, M
    
        N = n
        M = m
        
        SIMULACION()

        if len(PTO_GET) > 0:
            prom_ocioso_get = sum(PTO_GET) / len(PTO_GET)
        else:
            prom_ocioso_get = 0

        if len(PTO_POST) > 0:
            prom_ocioso_post = sum(PTO_POST) / len(PTO_POST)
        else:
            prom_ocioso_post = 0

        resultados.append({
            "N": N,
            "M": M,
            "PPTO_GET": prom_ocioso_get,
            "PPTO_POST": prom_ocioso_post,
            "TRP_Simples": TRP_Simples,
            "TRP_Complejas": TRP_Complejas
        })

        GENERAR_DATAFRAME()

        df_resultados = pd.DataFrame(resultados)
        print(df_resultados)

In [ ]:
RUN(5,19) # INTERMEDIO
RUN(2,18) # ACTUAL
RUN(3,20) # OPTIMO

   N   M  PPTO_GET  PPTO_POST  TRP_Simples  TRP_Complejas
0  5  19    72.202  31.151053          256           1603


C:\Users\nazar\AppData\Local\Temp\ipykernel_27492\3172131137.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_existente, nuevo_df], ignore_index=True)


   N   M  PPTO_GET  PPTO_POST  TRP_Simples  TRP_Complejas
0  5  19    72.202  31.151053          256           1603
1  2  18    31.020  27.258889         1598         210543
   N   M   PPTO_GET  PPTO_POST  TRP_Simples  TRP_Complejas
0  5  19  72.202000  31.151053          256           1603
1  2  18  31.020000  27.258889         1598         210543
2  3  20  53.663333  34.601000          295           1267


C:\Users\nazar\AppData\Local\Temp\ipykernel_27492\3172131137.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_existente, nuevo_df], ignore_index=True)


In [29]:
df_res = pd.read_excel("resultados.xlsx")

df_res = df_res.sort_values(by=["N", "M"], ascending=[True, True]).reset_index(drop=True)

In [31]:
display(df_res)

,N,M,TRP_Simples,TRP_Complejas,PTO_GET_1,PTO_GET_2,PTO_GET_3,PTO_GET_4,PTO_GET_5,PTO_POST_1,PTO_POST_2,PTO_POST_3,PTO_POST_4,PTO_POST_5,PTO_POST_6,PTO_POST_7,PTO_POST_8,PTO_POST_9,PTO_POST_10,PTO_POST_11,PTO_POST_12,PTO_POST_13,PTO_POST_14,PTO_POST_15,PTO_POST_16,PTO_POST_17,PTO_POST_18,PTO_POST_19,PTO_POST_20,tipo
0,2,18,1598,210543,25.66,36.38,NaN,NaN,NaN,5.58,6.57,7.88,9.44,11.36,13.66,16.39,19.67,23.38,27.53,31.62,35.86,39.84,43.31,46.32,49.10,50.74,52.41,NaN,NaN,INTERMEDIO
1,3,20,295,1267,38.54,53.98,68.47,NaN,NaN,6.80,7.87,9.28,10.96,13.10,15.64,18.89,22.27,26.60,31.08,35.73,40.42,44.91,49.00,52.84,56.11,58.94,61.52,63.90,66.16,OPTIMO
2,5,19,256,1603,43.41,60.39,76.01,87.25,93.95,6.19,7.29,8.72,10.48,12.46,14.95,18.10,21.69,25.69,29.96,34.34,38.88,43.15,46.80,50.24,53.24,55.18,57.33,57.18,NaN,ACTUAL
